In [ ]:
from scrapy import signals
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup as bs
import pandas as pd

class Driver():
    
    def __init__(self):
        options = webdriver.ChromeOptions()
        options.add_argument('user-data-dir="/Users/zhangming/Library/Application Support/Google/Chrome/Default"')
        options.add_argument('user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36"')
        self.driver = webdriver.Chrome(chrome_options=options)
    
    def get(self,url):
        self.driver.get(url)
        content = self.driver.page_source
        return content
    def status(self):
        return self.status
    
    def close(self):
        self.driver.close()
        
    def get_driver(self):
        return self.driver
         
    
    
class WebDriver():
    
    
    driver=[]
    url_queue=[]
    number=1
    hash_key=0

    def __init__(self,number=1):
        
        for x in range(number):
            
            dr=Driver()
            dr.get('https://www.taobao.com')
            self.driver.append(dr)            

        self.number=number
        
        
    def __run__(self):
        
        while(True):
            if len(url_queue)!=0:
                for x in range(number):
                    if self.driver[x].status:
                        self.driver[x].get(url_queue.pop())
                        break
            time.sleep(5)
            
    def close(self):
        for x in range(self.number):
            self.driver[x].close()
        
    
    def get(self,url):
        self.hash_key=self.hash_key+1
        return self.driver[self.hash_key%self.number].get(url)
    '''
        while(True):
            for x in range(self.number):
                if self.driver[x].status:
                    return self.driver[x].get(url)'''
        
        
    
    def __push(self,url):
        self.url_queue.append(url)
        
    
    def __pop(self):
        if len(self.url_queue)!=0:
            return self.url_queue.pop(0)
    

    def loadcookie(self,path):
        with open(path,'r') as f:
            text=f.read()
        
        items=text.split(';')
        ret={}
        for item in items:
            vals=item.split('=',maxsplit=1)
            ret[vals[0]]=vals[1]
            
        return ret   
        
    def setcookie(self,cookie):
        for driver in self.driver:
            driver.delete_all_cookies()
            for key,val in cookie.items():
                driver.add_cookie({'name':key,'value':val})
       
    def search(self,key):
        driver=self.driver[self.hash_key%self.number].get_driver()
        bar=driver.find_element_by_css_selector('input#q.search-combobox-input').clear()
        button=driver.find_element_by_css_selector('button.btn-search.tb-bg')
        ActionChains(driver).send_keys(key).click(button).perform()
        
    def input_to_search(self,key):
        driver=self.driver[self.hash_key%self.number].get_driver()
        bar=driver.find_element_by_css_selector('input#q.search-combobox-input').clear()
        button=driver.find_element_by_css_selector('button.submit.icon-btn-search')
        ActionChains(driver).send_keys(key).click(button).perform()
        return driver.page_source


In [ ]:
k=WebDriver()


In [ ]:
k.search('书')

In [ ]:
def searchurl(key):
    keys=key.split()
    key_str='+'.join(keys)
    return 'https://s.taobao.com/search?q='+key_str+'+书&imgfile=&js=1&stats_click=search_radio_all%3A1&initiative_id=staobaoz_20181012&ie=utf8'
    
  

In [ ]:

def loadkeys(path):
    data=pd.read_csv(path,header=None)
    return data.iloc[:,0].values

In [ ]:
import re
def parse(soup,key):
    output=[]
    for item in soup.find_all('div', class_=re.compile("item J_MouserOnverReq")):
        data_list=[]
        name=item.find('img',class_=re.compile('J_ItemPic'))['alt']
        price=item.find('a',class_=re.compile('J_ClickStat'))['trace-price']
        shop_name=item.find('div',class_=re.compile('shop')).get_text().strip()
        shop_url=item.find('a',class_=re.compile('shopname'))['href']
        item_url=item.find('a',class_=re.compile('pic-link'))['data-href']
    
        data_list.append(key)
        data_list.append(name)
        data_list.append(item_url)
        data_list.append(price)
        data_list.append(shop_name)
        data_list.append(shop_url)
        output.append(data_list)
    print(output)
    return output

In [ ]:
def run(driver):

    keys=loadkeys('booklist.csv')
    start_urls={}
    data_list=[]
    
    for key in keys:
        start_urls[key]=searchurl(key)
        #print(start_urls[key])
        
    for key, url in start_urls.items():
        time.sleep(10)
        content=driver.input_to_search(key)
        #content=driver.get(url)
        data=parse(bs(content,'lxml'),key)
        data_list.extend(data)

    return data_list  
        

In [ ]:
data_list=run(k)

In [ ]:
columns=['key','item','item_url','price','shop','shop_url']
csv_data=pd.DataFrame(data_list,columns=columns)
data=csv_data.to_csv('searchbook.csv')

In [ ]:
k.close()